In [1]:
import requests as req
from requests import Session
import json
import pandas as pd
from tqdm.auto import tqdm

In [2]:
df = pd.read_csv('codes_202301081543327.csv')
f = lambda x: x.split("|")[0].split("p=")[-1]
lista_nfs = df.text.map(f)

In [3]:
_CPF = None
_PASSWORD = None
_TOKEN = None
#CHAVE_ACESSO_NFCE = '11221234034195000102650020000136761010457960'
_DOAR_PARA_ENTIDADE_ID = '38'

In [4]:
def login(CPF: str, PASSWORD: str) -> Session:
    URI_LOGIN = 'https://api.notalegal.sefin.ro.gov.br/nota-legal-api/api/v1/auth/login'
    body = {'cpf': CPF, 'password': PASSWORD}
    s = Session()
    r = s.post(URI_LOGIN, body)
    response = json.loads(r.text)
    status = response.get('status', None)
    if status == 200:
        TOKEN = response['usuario']['token']
        s.headers.update({'Authorization': f"Bearer {TOKEN}"})
        print("Token retrieved! Session updated.")
    else:
        print(f'Token Not Retrieved. Status: {status}')
    return s


def consulta_nfce(s: Session, CHAVE_ACESSO_NFCE: str) -> dict:
    URI_CONSULTA_NFCE = f'https://api.notalegal.sefin.ro.gov.br/nota-legal-api/api/v1/nfce/consulta_nfce?chave_acesso_nfce={CHAVE_ACESSO_NFCE}'
    r = s.post(URI_CONSULTA_NFCE)
    if len(r.text) > 0:
        response = json.loads(r.text)
        if 'Cupom Fiscal localizado' in response.get('msg', ''):
            print(f"{response['msg']}, valor: {response['nfce']['VL-TOTAL']}, {response['nfce']['SITUACAO']}")
        else:
            print("Cupom fiscal não localizado. Inspecionar saida")
        return response
    else:
        print("Nenhuma resposta ao tentar consultar")

def doar_nfce(s: Session, CHAVE_ACESSO_NFCE: str, DOAR_PARA_ENTIDADE_ID: str) -> bool:
    URI_DOAR_NFCE = 'https://api.notalegal.sefin.ro.gov.br/nota-legal-api/api/v1/doacoes'
    body = {"doacao": {"chave_acesso": CHAVE_ACESSO_NFCE, "entidade_id": DOAR_PARA_ENTIDADE_ID}}
    r = s.post(URI_DOAR_NFCE, json=body)
    if len(r.text) > 0:
        response = json.loads(r.text)
        if response.get("status", None) == 200:
            print("NFCE doada")
            return True   
        else:
            print("NFCE já doada")
            return False
    print("Nenhuma resposta ao tentar doar")


def raspar_nfce(s: Session, CHAVE_ACESSO_NFCE: str):
    URI_RASPAR_NFCE = 'https://api.notalegal.sefin.ro.gov.br/nota-legal-api/api/v1/sorteio_raspadinha'
    body = {"chave_acesso_nfce": CHAVE_ACESSO_NFCE}
    r = s.post(URI_RASPAR_NFCE, json=body)
    if len(r.text) > 0:
        response = json.loads(r.text)
        msg = response.get('msgm', "")
        if "Não há" in msg:
            print("NFCE raspada - não ganhou")
        elif "O cupom já participou" in msg:
            print("NFCE não raspada - já participou")
        else:
            print(msg)
    else:
        print("Nenhuma resposta ao tentar raspar")

In [5]:
s = login(_CPF, _PASSWORD)
for nf in lista_nfs:
    print(nf)
    consulta_nfce(s, nf)
    doar_nfce(s, nf, _DOAR_PARA_ENTIDADE_ID)
    raspar_nfce(s, nf)
    print("---")

NFCE doada
NFCE raspada - não ganhou
---
11221234034195000102650020000131281077802053
Cupom Fiscal localizado., valor: 98.20, leitura_confirmada
NFCE doada
NFCE raspada - não ganhou
---
11221234034195000102650020000131271725310910
Cupom Fiscal localizado., valor: 48.90, leitura_confirmada
NFCE doada
NFCE raspada - não ganhou
---
11221234034195000102650020000131261892154890
Cupom Fiscal localizado., valor: 40.90, leitura_confirmada
NFCE doada
NFCE raspada - não ganhou
---
11221234034195000102650020000131251449368358
Cupom Fiscal localizado., valor: 40.90, leitura_confirmada
NFCE doada
NFCE raspada - não ganhou
---
11221234034195000102650020000131241114561345
Cupom Fiscal localizado., valor: 48.90, leitura_confirmada
NFCE doada
NFCE raspada - não ganhou
---
11221234034195000102650020000131221284629000
Cupom Fiscal localizado., valor: 39.90, leitura_confirmada
NFCE doada
NFCE raspada - não ganhou
---
11221234034195000102650020000131211559452516
Cupom Fiscal localizado., valor: 116.70, lei